# 21 Feb 2023
# nrobot

https://online.drl.wi.gov/orders/searchorders.aspx
 - Wisconsin Department of Safety and Professional Services
 - Health & Business Renewal Application 

In [1]:
# https://towardsdatascience.com/how-to-extract-text-from-pdf-245482a96de7

# install libraries

In [2]:
#!pip install pdfplumber
# !pip install pdfminer3 # old / unmaintained
# https://github.com/pdfminer/pdfminer.six
# !pip install pdfminer.six # also fails -- I realized that's because the 
# text of the order is actually scanned (image?)
#!pip install borb
# https://pyimagesearch.com/2021/08/16/installing-tesseract-pytesseract-and-python-ocr-packages-on-your-system/
#!pip install pytesseract
#!brew install freetype imagemagick
#!pip install pdf2image

# try various things

In [3]:
# only extracts the first (easy) page 
import pdfplumber 
with pdfplumber.open(r'./ORDER0008271-00019637.pdf') as pdf:
    first_page = pdf.pages[0]
    #print(first_page.extract_text())

ModuleNotFoundError: No module named 'pdfplumber'

In [4]:
from pdfminer.high_level import extract_text

text = extract_text("./ORDER0008271-00019637.pdf")
#print(text)

In [5]:
# pdf info: created with www.pdfsharp.com

In [6]:
# https://stackabuse.com/applying-ocr-to-a-scanned-pdf-in-python-using-borb/

In [7]:
# New imports
import typing
from pathlib import Path

from borb.pdf import Document
from borb.pdf import Page
from borb.toolkit import SimpleTextExtraction
#from borb.pdf import SingleColumnLayout
#from borb.pdf import Paragraph
from borb.pdf import PDF

def read_modified_document():

    # read the Document
    doc: typing.Optional[Document] = None
    l: SimpleTextExtraction = SimpleTextExtraction()

    with open("./ORDER0008271-00019637.pdf", "rb") as in_file_handle:
        #doc = PDF.loads(in_file_handle)
        doc = PDF.loads(in_file_handle, [l])

    # check whether we have read a Document
    assert doc is not None

    # print the \Producer key from the \Info dictionary
    print("Producer: %s" % doc.get_document_info().get_producer())
    print(f'Num. of pages: {len(l.get_text())}')
    print(l.get_text()[0][:500])
    #print(f'# of Pages: {l.shape}')
    return l

l = read_modified_document()


Producer: PDFsharp 1.31.1789-g \(www.pdfsharp.com\)
Num. of pages: 1
WISCONSIN DEPARTMENT OF
SAFETY AND PROFESSIONAL SERVICES
Wisconsin Department of Safety and Professional Services
Access to the Public Records of the Reports of Decisions
This Reports of Decisions document was retrieved from the Wisconsin Department of Safety and
Professional Services website. These records are open to public view under Wisconsin’s Open
Records law, sections 19.31-19.39 Wisconsin Statutes.
Please read this agreement prior to viewing the Decision:
 The Reports of Decisions is de


In [8]:
# https://github.com/jorisschellekens/borb-examples#72-performing-ocr-on-a-pdf

In [9]:
!brew info tesseract

==> tesseract: stable 5.3.0 (bottled), HEAD
OCR (Optical Character Recognition) engine
https://github.com/tesseract-ocr/
/opt/homebrew/Cellar/tesseract/5.3.0_1 (73 files, 32.2MB) *
  Poured from bottle on 2023-01-23 at 18:01:00
From: https://github.com/Homebrew/homebrew-core/blob/HEAD/Formula/tesseract.rb
License: Apache-2.0
==> Dependencies
Build: autoconf ✔, automake ✔, libtool ✔, pkg-config ✔
Required: cairo ✔, icu4c ✔, leptonica ✔, libarchive ✔, pango ✔
==> Options
--HEAD
	Install HEAD version
==> Caveats
This formula contains only the "eng", "osd", and "snum" language data files.
If you need any other supported languages, run `brew install tesseract-lang`.
==> Analytics
install: 175,981 (30 days), 465,732 (90 days), 1,465,418 (365 days)
install-on-request: 66,153 (30 days), 146,394 (90 days), 345,527 (365 days)
build-error: 10 (30 days)


In [10]:
# TesseractError: (1, 'Error opening data file /opt/homebrew/Cellar/tesseract/5.3.0_1/share/eng.traineddata Please make sure the TESSDATA_PREFIX environment variable is set to your "tessdata" directory. Failed loading language \'eng\' Tesseract couldn\'t load any languages! Could not initialize tesseract.')

# > Fix with brew info tesseract + share + tessdata, as below

In [11]:
!ls /opt/homebrew/Cellar/tesseract/5.3.0_1/share/tessdata

configs          osd.traineddata  snum.traineddata
eng.traineddata  pdf.ttf          tessconfigs


In [12]:
from borb.pdf import Document
from borb.pdf import PDF
from borb.toolkit.ocr.ocr_as_optional_content_group import OCRAsOptionalContentGroup

from pathlib import Path

# set up everything for OCR
tesseract_data_dir: Path = Path("/opt/homebrew/Cellar/tesseract/5.3.0_1/share/tessdata")
assert tesseract_data_dir.exists()
#l: OCRAsOptionalContentGroup = OCRAsOptionalContentGroup(tesseract_data_dir)
l: SimpleTextExtraction = SimpleTextExtraction()

# read Document
doc: typing.Optional[Document] = None
#with open("ORDER0008271-00019637.pdf", "rb") as pdf_file_handle:
with open("page3.pdf", "rb") as pdf_file_handle:
    doc = PDF.loads(pdf_file_handle, [l])

assert doc is not None
len(l.get_text())
l.get_text()


{}

In [ ]:
# https://stackoverflow.com/questions/45480280/convert-scanned-pdf-to-text-python

In [ ]:
# https://stackoverflow.com/questions/45480280/convert-scanned-pdf-to-text-python
import glob
from pdf2image import convert_from_path

pdfs = glob.glob(r"page3.pdf")

for pdf_path in pdfs:
    pages = convert_from_path(pdf_path, 500)

    for pageNum,imgBlob in enumerate(pages):
        text = pytesseract.image_to_string(imgBlob,lang='eng')

        with open(f'{pdf_path[:-4]}_page{pageNum}.txt', 'w') as the_file:
            the_file.write(text)

ModuleNotFoundError: No module named 'pdf2image'

# try Imagemagick OCR

In [4]:
# ImportError: MagickWand shared library not found.
# You probably had not installed ImageMagick library.
# cannot find libwand, libmagick
# https://stackoverflow.com/a/71591742/851192 

In [5]:
!brew info imagemagick


/bin/bash: line 1: brew: command not found


In [6]:
#!echo 'export MAGICK_HOME=/opt/homebrew/Cellar/imagemagick/7.1.0-62/' >> ~/.bash_profile
#!echo 'export PATH="$MAGICK_HOME/bin:$PATH"' >> ~/.bash_profile
#!echo 'export PATH="/opt/homebrew/Cellar/imagemagick/7.1.0-62/bin:$PATH"' >> ~/.bash_profile
!echo $PATH | grep brew


##  !--- current

In [3]:
!echo $PATH


/home/chai/v3/bin:/home/chai/v3/bin:/home/chai/.vscode-server/bin/92d25e35d9bf1a6b16f7d0758f25d48ace11e5b9/bin/remote-cli:/home/chai/.local/bin:/home/chai/.local/bin:/home/chai/miniconda3/bin:/home/chai/miniconda3/condabin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/snap/bin:/home/chai/.local/bin:/home/chai/.local/bin


In [4]:
import wand.version
print(wand.version.MAGICK_VERSION)

ImageMagick 6.9.11-60 Q16 x86_64 2021-01-25 https://imagemagick.org


In [5]:
PATH="$MAGICK_HOME/bin:$PATH"
PATH="/opt/homebrew/Cellar/imagemagick/7.1.0-62/bin:$PATH"

In [41]:
import os
import io
from PIL import Image as pImg 
import pytesseract
from wand.image import Image as wandImg 
import gc

#pdf_path = './page3.pdf'

#pdf = wandImg(filename=pdf_path,resolution=300)
#pdfImg=pdf.convert('jpeg')

imgBlobs=[]
extracted_text=[]

def get_text_from_image(pdf_path):
    pdf = wandImg(filename=pdf_path,resolution=300)
    pdf_JPEG = pdf.convert('jpeg')
    imgBlobs = []
    extracted_text = []

    if True:
        n = 1 # skip first page
    else:
        n = 0

    for img in pdf_JPEG.sequence[n:]:
        page = wandImg(image=img)
        imgBlobs.append(page.make_blob('jpeg'))

    #print(len(imgBlobs))
    for imgBlob in imgBlobs:
        img = pImg.open(io.BytesIO(imgBlob))
        text=pytesseract.image_to_string(img,lang='eng')
        extracted_text.append(text)

    return (extracted_text)

In [45]:
print(f'{pytesseract.get_tesseract_version()=}')

pytesseract.get_tesseract_version()=<Version('5.1.0')>


In [47]:
import glob
fname = sorted(glob.glob('./wisconsin_pdfs/*.pdf'))[0]
print(fname)
get_text_from_image(fname)
from datetime import datetime
datetime.now() 

./wisconsin_pdfs/Massage Therapist Or Bodyworker - WI - ORDER0002114-00007904.pdf


datetime.datetime(2023, 2, 27, 16, 7, 2, 236033)

In [48]:
#!mkdir 

In [49]:
# get_text_from_image('./wisconsin_pdfs/Massage Therapist or Bodywork Therapist - WI - ORDER0005767-00014684.pdf)

'''
print('5668')
get_text_from_image('./wisconsin_pdfs/Massage Therapist or Bodywork Therapist - WI - ORDER0005668-00014483.pdf')
print('5767')
get_text_from_image('./wisconsin_pdfs/Massage Therapist or Bodywork Therapist - WI - ORDER0005767-00014684.pdf')
print('5768')
get_text_from_image('./wisconsin_pdfs/Massage Therapist or Bodywork Therapist - WI - ORDER0005768-00014685.pdf')
'''

5668
5767


KeyboardInterrupt: 

In [61]:
#!ls ./wisconsin_pdfs

In [70]:
import glob
from pathlib import Path
import os

files = sorted(glob.glob('./wisconsin_pdfs/*.pdf'))#, key=os.path.getmtime)
folder = './wisconsin_pdfs'
files = [
    'Massage Therapist or Bodywork Therapist - WI - ORDER0007990-00019037.pdf',
    'Massage Therapist or Bodywork Therapist - WI - ORDER0002000-00008174.pdf',
    'Massage Therapist or Bodywork Therapist - WI - ORDER0005767-00014684.pdf',
    'Massage Therapist or Bodywork Therapist - WI - ORDER0007403-00018082.pdf',
    'Massage Therapist or Bodywork Therapist - WI - ORDER0007700-00018417.pdf',
    'Massage Therapist or Bodywork Therapist - WI - ORDER0007990-00019037.pdf',
    'Massage Therapist or Bodywork Therapist - WI - ORDER0008135-00019340.pdf', ]
files = [os.path.join(folder, fname) for fname in files]

for i, file in enumerate(files):
    print(f'{i} - {file}')
    start_time = datetime.now()

    out_fname = Path(file).stem + '.txt'
    #if not os.path.exists(out_fname):
    with open(out_fname, 'w') as outfile:
        outfile.writelines(get_text_from_image(file))

    elapsed_time = datetime.now() - start_time
    print(f'{elapsed_time}')

0 - ./wisconsin_pdfs/Massage Therapist or Bodywork Therapist - WI - ORDER0007990-00019037.pdf
0:00:06.704592
1 - ./wisconsin_pdfs/Massage Therapist or Bodywork Therapist - WI - ORDER0002000-00008174.pdf
0:00:01.578940
2 - ./wisconsin_pdfs/Massage Therapist or Bodywork Therapist - WI - ORDER0005767-00014684.pdf
0:00:05.485315
3 - ./wisconsin_pdfs/Massage Therapist or Bodywork Therapist - WI - ORDER0007403-00018082.pdf
0:00:05.620855
4 - ./wisconsin_pdfs/Massage Therapist or Bodywork Therapist - WI - ORDER0007700-00018417.pdf
0:00:05.091522
5 - ./wisconsin_pdfs/Massage Therapist or Bodywork Therapist - WI - ORDER0007990-00019037.pdf
0:00:06.750492
6 - ./wisconsin_pdfs/Massage Therapist or Bodywork Therapist - WI - ORDER0008135-00019340.pdf
0:00:08.205877


In [ ]:
# /etc/ImageMagick-6/policy.xml.
# !mv wi_decisions/*.txt wi_decisions/ocr_output_txt/

In [ ]:
!ls -lahS